In [1]:
from IPython import display
from ultralytics import YOLO
import supervision as sv
import cv2
import numpy as np
from datetime import timedelta
import os
from matplotlib import rcParams
from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib.pyplot as plt
from PIL import Image
display.clear_output()

In [2]:
def get_video_info(video_path):
    
    # Extracting information about the video
    video_info = sv.VideoInfo.from_video_path(video_path)
    width, height, fps, total_frames = video_info.width, video_info.height, video_info.fps, video_info.total_frames
    
    # Calculating the length of the video by dividing the total number of frames by the frame rate and rounding to the nearest second
    video_length = timedelta(seconds = round(total_frames / fps))
    
    # Print out the video resolution, fps, and length u
    print(f"\033[1mVideo Resolution:\033[0m ({width}, {height})")
    print(f"\033[1mFPS:\033[0m {fps}")
    print(f"\033[1mLength:\033[0m {video_length}")

In [3]:
get_video_info('ref_vid.mp4')

Video Resolution: (2592, 1944)
FPS: 30
Length: 0:00:24


In [4]:
def person_count(source_path,line_start, line_end):
   
   # Load the pre-trained YOLO model
   model = YOLO('yolov8x.pt')

   # Create two points from the line_start and line_end coordinates
   line_start = sv.Point(line_start[0], line_start[1])
   line_end = sv.Point(line_end[0], line_end[1])
   
   # Create a line zone object from the two points
   line_counter = sv.LineZone(line_start, line_end) 
   
   # Create a line zone annotator object with specific thickness and text scale
   line_annotator = sv.LineZoneAnnotator(thickness = 2,
                                         text_thickness = 1,
                                         text_scale = 0.5)
   
   # Create a box annotator object with specific thickness and text scale
   box_annotator = sv.BoxAnnotator(thickness = 1,
                                   text_thickness = 1,
                                   text_scale = 0.5)
   
   # Extract information about the video from the given source path
   video_info = sv.VideoInfo.from_video_path(source_path)

   # Create a video out path by combining the destination path and the video name
   video_name = os.path.splitext(os.path.basename(source_path))[0] + ".mp4"
   video_out_path = "ref_vid_detected.mp4"

   # Create a video writer object for the output video
   video_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'mp4v'), video_info.fps, (video_info.width, video_info.height))

   # Loop over each frame of the video and perform object detection and tracking
   for result in model.track(source = source_path, tracker = 'botsort.yaml', conf=0.01, show=True, stream=True, agnostic_nms=True):
        

        
      
        # Get the original frame from the detection result
        frame = result.orig_img
        
        # Convert the YOLO detection results to a Detections object
        detections = sv.Detections.from_ultralytics(result)

        if detections.tracker_id is not None:

          detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

          # Filter the detections
          detections = detections[detections.class_id == 0]

          # labels = [
          #             f"{model.model.names[class_id]} confidence:{confidence}" 
          #             for _, _, confidence, class_id, _, _ in detections
          #         ]

          labels = [
                      f"{tracker_id} confidence:{confidence}" 
                      for _, _, confidence, class_id, tracker_id, _ in detections
                  ]

          # Trigger the line counter to count any detections that intersect the line zone
          line_counter.trigger(detections)
      
          # Annotate the frame with the line zone and any intersecting detections
          line_annotator.annotate(frame, line_counter)

          # Annotate the frame with bounding boxes and labels for all detections
          frame = box_annotator.annotate(scene = frame,
                                        detections = detections,
                                        labels = labels)
          

          # Write the annotated frame to the output video
          video_out.write(frame)
  
   # Release the video writer and clear the Jupyter Notebook output
   video_out.release()
   display.clear_output()


# Performing vehicle count on the test video
person_count(source_path = "ref_vid.mp4",
              line_start = (337, 391),
              line_end = (917, 387))

: 